# Sage Simplex Implementations

In [9]:
import numpy as np

## ``GLPK/exact`` backend

In [18]:
p = MixedIntegerLinearProgram(maximization=False, solver = "GLPK/exact")
w = p.new_variable(nonnegative=True)
p.add_constraint(w[0] + w[1] + w[2] - 14*w[3] == 0)
p.add_constraint(w[1] + 2*w[2] - 8*w[3] == 0)
p.add_constraint(2*w[2] - 3*w[3] == 0)
p.add_constraint(w[0] - w[1] - w[2] >= 0)
p.add_constraint(w[3] >= 1)
p.set_objective(w[3])
p.show()

Minimization:
  x_3 

Constraints:
  0.0 <= x_0 + x_1 + x_2 - 14.0 x_3 <= 0.0
  0.0 <= x_1 + 2.0 x_2 - 8.0 x_3 <= 0.0
  0.0 <= 2.0 x_2 - 3.0 x_3 <= 0.0
  - x_0 + x_1 + x_2 <= 0.0
  - x_3 <= -1.0
Variables:
  x_0 is a continuous variable (min=0.0, max=+oo)
  x_1 is a continuous variable (min=0.0, max=+oo)
  x_2 is a continuous variable (min=0.0, max=+oo)
  x_3 is a continuous variable (min=0.0, max=+oo)


In [9]:
p.solve()

1

In [10]:
p2 = MixedIntegerLinearProgram(maximization=True, solver = "GLPK/exact")
x = p2.new_variable(nonnegative=True)
p2.set_objective(19 * x[0] + 13 * x[1] + 12 * x[2] + 17 * x[3])
p2.add_constraint(3 * x[0] + 2 * x[1] + x[2] + 2 * x[3] <= 225)
p2.add_constraint(x[0] + x[1] + x[2] + x[3] <= 117)
p2.add_constraint(4 * x[0] + 3 * x[1] + 3 * x[2] + 4 * x[3] <= 420)
p2.show()

Maximization:
  19.0 x_0 + 13.0 x_1 + 12.0 x_2 + 17.0 x_3 

Constraints:
  3.0 x_0 + 2.0 x_1 + x_2 + 2.0 x_3 <= 225.0
  x_0 + x_1 + x_2 + x_3 <= 117.0
  4.0 x_0 + 3.0 x_1 + 3.0 x_2 + 4.0 x_3 <= 420.0
Variables:
  x_0 is a continuous variable (min=0.0, max=+oo)
  x_1 is a continuous variable (min=0.0, max=+oo)
  x_2 is a continuous variable (min=0.0, max=+oo)
  x_3 is a continuous variable (min=0.0, max=+oo)


In [11]:
p2.solve()

1827.0

In [12]:
two_phase = MixedIntegerLinearProgram(maximization=True, solver = "GLPK")
x = two_phase.new_variable(nonnegative=True)
slack = two_phase.new_variable(nonnegative=True)
two_phase.set_objective(x[0] - x[1] + x[2])
two_phase.add_constraint(2 * x[0] - x[1] + 2 * x[2] + slack[0] == 4)
two_phase.add_constraint(2 * x[0] - 3 * x[1] + x[2] + slack[1] == -5)
two_phase.add_constraint(-x[0] + x[1] - 2 * x[2] + slack[2] == -1)
two_phase.show()

Maximization:
  x_0 - x_1 + x_2 

Constraints:
  4.0 <= 2.0 x_0 - x_1 + 2.0 x_2 + x_3 <= 4.0
  -5.0 <= 2.0 x_0 - 3.0 x_1 + x_2 + x_4 <= -5.0
  -1.0 <= - x_0 + x_1 - 2.0 x_2 + x_5 <= -1.0
Variables:
  x_0 is a continuous variable (min=0.0, max=+oo)
  x_1 is a continuous variable (min=0.0, max=+oo)
  x_2 is a continuous variable (min=0.0, max=+oo)
  x_3 is a continuous variable (min=0.0, max=+oo)
  x_4 is a continuous variable (min=0.0, max=+oo)
  x_5 is a continuous variable (min=0.0, max=+oo)


In [13]:
np.isclose(two_phase.solve(), 3/5)

True

In [14]:
backend = two_phase.get_backend()
backend

In [15]:
backend.get_col_dual(2)

0.0

In [16]:
backend.solve()
backend.get_objective_value()

0.6000000000000001

In [17]:
backend.base_ring()

Real Double Field

In [18]:
[ backend.is_variable_basic(i) for i in range(backend.ncols()) ]

[False, True, True, False, False, True]

### Find initial basis

In [19]:
aux = MixedIntegerLinearProgram(solver = "GLPK/exact")
x = aux.new_variable(nonnegative=True)
aux.set_objective(0)
aux.add_constraint(2 * x[0] - x[1] + 2 * x[2] <= 4)
aux.add_constraint(2 * x[0] - 3 * x[1] + x[2] <= -5)
aux.add_constraint(-x[0] + x[1] - 2 * x[2] <= -1)
aux.show()

Maximization:
  

Constraints:
  2.0 x_0 - x_1 + 2.0 x_2 <= 4.0
  2.0 x_0 - 3.0 x_1 + x_2 <= -5.0
  - x_0 + x_1 - 2.0 x_2 <= -1.0
Variables:
  x_0 is a continuous variable (min=0.0, max=+oo)
  x_1 is a continuous variable (min=0.0, max=+oo)
  x_2 is a continuous variable (min=0.0, max=+oo)


In [20]:
aux_backend = aux.get_backend()
aux_backend.solve()
[ aux_backend.is_variable_basic(i) for i in range(aux_backend.ncols()) ]

[False, True, True]

## Interactive Simplex Method

In [21]:
m = matrix(QQ,[[1,2,3],[4,5,6]])
m

[1 2 3]
[4 5 6]

In [4]:
from pathlib import Path
from mcf_simplex_analyzer.load_instance import load_instance

# Path to data directory
instances_path = Path("example/")

assert instances_path.exists()

instance_format = "planar"
nod_file = instances_path / (instance_format + ".nod")
arc_file = instances_path / (instance_format + ".arc")
sup_file = instances_path / (instance_format + ".sup")
mut_file = instances_path / (instance_format + ".mut")

instance = load_instance(instance_format, nod_file, arc_file, sup_file, mut_file)
print(instance.info)

InstanceInfo(products_no=92, nodes_no=30, links_no=150, bundled_links_no=150)


In [5]:
from mcf_simplex_analyzer.formulate import formulate_concurrent_flow_problem

network_info, max_flow_sum = formulate_concurrent_flow_problem(instance)

In [6]:
def solve_for(instance, network_info, y=1, product_limit=infinity, solver="GLPK/exact"):
    model = MixedIntegerLinearProgram(maximization=False, solver=solver)
    
    flow = model.new_variable(nonnegative=True)
    violation = model.new_variable(nonnegative=True)

    # Objective
    model.set_objective(model.sum(
            violation[(u, v)]
            for u in network_info.capacities
            for v in network_info.capacities[u]
        
    ))

    # Edges
    for u in network_info.capacities:
        for v in network_info.capacities[u]:
            model.add_constraint(
                model.sum(
                    
                        flow[(u, v, commodity)]
                        for commodity in range(1, min(instance.info.products_no + 1, product_limit))
                    
                )
                - violation[(u, v)]
                <= network_info.capacities[u][v]
            )

    vertices = set()
    for u in network_info.capacities:
        for v in network_info.capacities[u]:
            vertices.add(u)
            vertices.add(v)

    # Kirchhof
    for commodity in range(1, min(instance.info.products_no + 1, product_limit)):
        for vertex in vertices:
            if vertex in network_info.sources and commodity in network_info.sources[vertex]:
                continue

            if (
                vertex in network_info.destinations
                and commodity in network_info.destinations[vertex]
            ):
                continue

            outgoing = model.sum(
                    flow[(vertex, u, commodity)]
                    for u in network_info.capacities.get(vertex, [])
            )

            incomming = model.sum(
                    flow[(u, vertex, commodity)]
                    for u in network_info.capacities
                    if vertex in network_info.capacities.get(u, {})
            )
            model.add_constraint(outgoing - incomming == 0)

    for destination in network_info.destinations:
        for commodity in network_info.destinations[destination]:
            incomming = model.sum(
                    
                        flow[(u, destination, commodity)]
                        for u in network_info.capacities
                        if destination in network_info.capacities.get(u, {})
                    
                )
            model.add_constraint( incomming >= y * network_info.destinations[destination][commodity].numerator)
            
    return model

In [ ]:
import time

data = {}
solvers = ["GLPK", "GLPK/exact", "InteractiveLP"]
for solver in solvers:
    
    data[solver] = []
    with open(solver.replace('/', '') + ".benchmark", 'w') as fout:
        print(solver)
        print('', 'LIM', 'NVARS', 'TTF', 'TTS', '%TTF', sep='\t')
        for limit in range(1, 10):
            # Measure time to formulate
            tic = time.perf_counter()
            model = solve_for(instance, network_info, y=20, product_limit=limit, solver=solver)
            toc = time.perf_counter()
            time_to_formulate = toc - tic
            
            # Measure time to solve
            tic = time.perf_counter()
            out = model.solve()
            toc = time.perf_counter()
            time_to_solve = toc - tic

            data[solver].append((limit, model.number_of_variables(), time_to_formulate, time_to_solve, (time_to_formulate + time_to_solve) / time_to_solve))
            print('', *data[solver][-1], sep='\t', flush=True)
            fout.write(','.join(map(str, data[solver][-1])) + '\n')

GLPK
	LIM	NVARS	TTF	TTS	%TTF
	1	623	0.015655797004001215	0.0012781200057361275	13.249082193955902
	2	768	0.010975074997986667	0.0020101760019315407	6.459758243776128
	3	915	0.013719121008762158	0.002973826995003037	5.613288208027766
	4	1059	0.014567692996934056	0.003436434009927325	5.239188925162028
	5	1207	0.01738462499633897	0.0039076999964891	5.4488126038227875
	6	1350	0.01806392399885226	0.004596067999955267	4.930299551492292
	7	1493	0.020810155998333357	0.005440518987597898	4.82503140707197
	8	1640	0.02163652300077956	0.005883570993319154	4.677447425270813
	9	1783	0.023396390010020696	0.007292086011148058	4.208463253759289
GLPK/exact
	LIM	NVARS	TTF	TTS	%TTF
	1	623	0.009174906997941434	0.01672277200850658	1.5486474965558532
	2	768	0.01174063301004935	0.023474813002394512	1.5001374455614094
	3	915	0.01303545200789813	0.03739459998905659	1.348591829079945
	4	1059	0.014718614998855628	0.0455829449929297	1.3228974126427901
	5	1207	0.016586532001383603	0.05714300899126101	1.290263538693

In [ ]:
import matplotlib.pyplot as plt
import csv

solvers = ["GLPK", "GLPK/exact", "InteractiveLP"]
for solver in solvers:
    with open(solver.replace('/', '') + ".benchmark", 'r') as fin:
        print(fin.read())
        

fig, axs = plt.subplots(2, 1, figsize=(10, 10))

axs[0].plot([ nvars for _, nvars, _, tts, _ in data["GLPK"]], [ tts / nvars for _, nvars, _, tts, _ in data["GLPK"]])
axs[1].plot([ nvars for _, nvars, _, tts, _ in data["GLPK/exact"]], [ tts / nvars for _, nvars, _, tts, _ in data["GLPK/exact"]])
axs[2].plot([ nvars for _, nvars, _, tts, _ in data["InteractiveLP"]], [ tts / nvars for _, nvars, _, tts, _ in data["InteractiveLP"]])

In [17]:
model = solve_for(instance, network_info, y=20, solver="GLPK")
backend = model.get_backend()
backend.solve()
basis = np.fromiter( (backend.is_variable_basic(i) for i in range(backend.ncols())) , dtype=bool)